In [2]:
import os
import sys
import random
import json
import glob
import tempfile
sys.path.append('../..')

import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
import tqdm

import cr_interface as cri
import cr_analysis as cra
import keras_utils as ku
import keras_bottle as kb

Using TensorFlow backend.


In [3]:
app = list(ku.applications.values())[0]
kb.get_bottleneck_dir(app.codename, app.get_model())

loading mobilenet model


'/Users/release/.ho/cardiac-research/slice_range/bottlenecks/MOB_conv_pw_13_relu'

In [13]:
a = np.array([[1, 2], [2, 3]])
b = np.array([[1, 2], [3, 4]])
np.stack([a, b])

array([[[1, 2],
        [2, 3]],

       [[1, 2],
        [3, 4]]])

In [29]:
def load_bottlenecks(cr_codes, model_codename, model,
                     augmented=False, multiplier=5, generate=False):
    bottle_dir = kb.get_bottleneck_dir(model_codename, model)
    bottle_paths = []
    
    if augmented:
        for i in range(multiplier):
            for cr in cr_codes:
                bottle_paths.append(os.path.join(
                    bottle_dir, '{}_AUG_{}.npy'.format(cr, i)))
    else:
        for cr in cr_codes:
            bottle_paths.append(os.path.join(
                bottle_dir,'{}.npy'.format(cr)))
    
    loaded = 0
    for path in bottle_paths:
        if os.path.exists(path):
            loaded += 1
            
    bottles = []
    if loaded == len(bottle_paths):
        print('loading bottlenecks...')
        for path in tqdm.tqdm(bottle_paths):
            bottles.append(np.load(path))
        return np.stack(bottles)
    else:
        if generate:
            raise NotImplementedError('image-wise generation not implemented')
        else:
            print('only generated {} of {} images'.format(
                loaded, len(bottle_paths)))

In [41]:
app = list(ku.applications.values())[0]
cr_codes = cri.extract_cr_codes(glob.glob(os.path.join(cri.DATABASE_DIR, '*.jpg')))
ho = load_bottlenecks(cr_codes, app.codename, app.get_model())
if ho is not None:
    ho.shape

  0%|          | 6/5057 [00:00<01:25, 58.90it/s]

loading bottlenecks...


100%|██████████| 5057/5057 [00:16<00:00, 313.67it/s]
